## Reading in data available through The Arabidopsis Information Resource (TAIR) database
The purpose of this notebook is to read in and do a preliminary survey of the data related to text and ontology annotations of text that is available through TAIR. The datasets need to be organized and also restructured into a standard format that will allow it be combined with datasets from other resources.

### Files read
```
phenologs-with-oats/data/gene_related_files/tair/Locus_Germplasm_Phenotype_20180702.txt
phenologs-with-oats/data/gene_related_files/tair/po_anatomy_gene_arabidopsis_tair.assoc
phenologs-with-oats/data/gene_related_files/tair/po_temporal_gene_arabidopsis_tair.assoc
phenologs-with-oats/data/gene_related_files/tair/ATH_GO_GOSLIM.txt
```

### Files created
```
phenologs-with-oats/data/reshaped_files/ath_phenotypes.csv
phenologs-with-oats/data/reshaped_files/ath_all_go_annotations.csv
phenologs-with-oats/data/reshaped_files/ath_high_confidence_go_annotations.csv
phenologs-with-oats/data/reshaped_files/ath_high_confidence_po_annotations.csv
```

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
import os
import warnings
import pandas as pd
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

sys.path.append("../../oats")
from oats.utils.constants import NCBI_TAG
from oats.utils.constants import EVIDENCE_CODES
from oats.utils.utils import to_abbreviation
from oats.nlp.preprocess import concatenate_with_bar_delim
from oats.nlp.preprocess import other_delim_to_bar_delim
from oats.nlp.preprocess import remove_punctuation
from oats.nlp.preprocess import remove_enclosing_brackets
from oats.nlp.preprocess import concatenate_descriptions
from oats.nlp.preprocess import add_prefix

OUTPUT_DIR = "../data/reshaped_files"
mpl.rcParams["figure.dpi"] = 200
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### File with genes and phenotype descriptions (Locus_Germplasm_Phenotype_20180702.txt)
Reading in the dataset of phenotypic descriptions. There is only one value specified as the gene name (locus name) in the original dataset so this column does not need to be parsed further. The descriptions commmonly use semi-colons to separate phrases. The next cell gets the distribution of the number of phrases in each description field for the dataset of text descriptions, as determined by a sentence parser. The majority of the descriptions are a single sentence or phrase, but some contain more.

In [ ]:
filename = "../data/gene_related_files/tair/Locus_Germplasm_Phenotype_20180702.txt"
usecols = ["LOCUS_NAME", "PHENOTYPE", "PUBMED_ID"]
usenames = ["gene_names", "description"]
renamed = {k:v for k,v in zip(usecols,usenames)}
df = pd.read_table(filename, usecols=usecols)
df.rename(columns=renamed, inplace=True)
df.sample(20)

In [ ]:
# Plotting distributions of number of phrases in each description.
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title("Phenotype Descriptions")
ax2.set_title("Phenotype Descriptions")
ax1.set_xlabel("Number of phrases")
ax2.set_xlabel("Number of words")
x1 = [len(sent_tokenize(x)) for x in df["description"].values]
x2 = [len(word_tokenize(x)) for x in df["description"].values]
ax1.hist(x1, bins=15, range=(0,15), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
ax2.hist(x2, bins=30, range=(0,200), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
fig.set_size_inches(15,4)
fig.tight_layout()
fig.show()

In [ ]:
# Restructuring the dataset to include all expected column names.
df["species"] = "ath"
df["term_ids"] = ""
df = df[["species", "gene_names", "description", "term_ids"]]

# Outputting the dataset of phenotype descriptions to csv file.
path = os.path.join(OUTPUT_DIR,"ath_phenotypes.csv")
df.to_csv(path, index=False)

### File with gene ontology annotations (ATH_GO_GOSLIM.txt)
Read in the file containing names of loci and corresponding information relating to gene ontology term annotation. Not all of the columns are used here, only a subset of them are read in. The relationship column refers to the relationships between the gene for that loci and the term mentioned on that given line. Evidence refer to the method of acquiring and the confidence in the annotation itself. This is retained so that we can subset that dataset based on whether the annotations are experimentally confirmed or simply predicted annotations. This section also looks at how many unique values are present for each field.

Each term annotation in this dataset is also associated with an evidence code specifying the method by which this annotation was made, which is related to the confidence that we can have in this annotation, and the tasks that the annotation should be used for. About half of the term annotations were made computationally, but there are also a high number of annotations available from high confidence annotations such as experimentally validated, curator statements, and author statements.

In [ ]:
filename = "../data/gene_related_files/tair/ATH_GO_GOSLIM.txt"
df_go = pd.read_table(filename, header=None, usecols=[0,3,4,5,9,12])
df_go.columns = ["locus","relationship","term_label","term_id","evidence_code","reference"]
unique_values = {col:len(pd.unique(df_go[col].values)) for col in df_go.columns}
print(df_go[["locus","term_id","evidence_code","reference"]].head(10))
print(df_go.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

In [ ]:
code_quantities = {c:len([x for x in df_go["evidence_code"] if EVIDENCE_CODES[x] in c]) 
             for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

In [ ]:
# Restructuring the dataset to include all the expected column names.
df_go["species"] = "ath"
df_go["gene_names"] = df_go["locus"]
df_go["description"] = ""
df_go["term_ids"] = df_go["term_id"]
high_confidence_categories = ["experimental","other_statement","curator_statement"]
df_go["high_confidence"] = df_go["evidence_code"].apply(lambda x: EVIDENCE_CODES[x] in high_confidence_categories)

# Subset to only include the high-quality GO annotations and ouptut the dataset to a csv file.
df_go_high_confidence = df_go[df_go["high_confidence"]==True]
path = os.path.join(OUTPUT_DIR,"ath_high_confidence_go_annotations.csv")
df_go_high_confidence = df_go_high_confidence[["species", "gene_names", "description", "term_ids"]]
df_go_high_confidence.to_csv(path, index=False)

# Outputting the dataset of annotations to a csv file.
df_go = df_go[["species", "gene_names", "description", "term_ids"]]
path = os.path.join(OUTPUT_DIR,"ath_all_go_annotations.csv")
df_go.to_csv(path, index=False)

### File with plant ontology term annotations (po_[termporal/anatomy]_gene_arabidopsis_tair.assoc)
There are two separate files available that include annotations of PO terms. The files do not have headers so column names are added based on how the columns are described in the accompanying available readme files. One of the files contains annotations for PO terms that are spatial, or describe a specific part of plant anatomy or plant molecular structures. The other file contains annotations for PO terms that are temporal, or refer to a specific process or stage of development. These files are each read in separately, and the next cells look at the quantity of unique values in the columns of each dataset. There are more spatial annotations than temporal annotations, and a greater number of terms used to describe the spatial annotations.

The next field combines the two datasets of PO annotations and looks at the number of unique values for each column in the resulting dataset. Because there is no overlap in the terms between the two, the datasets are simply appended to one another and the total unique terms are a sum of the individual datasets.

Each term annotation in this dataset is also associated with an evidence code specifying the method by which this annotation was made, which is related to the confidence that we can have in this annotation, and the tasks that the annotation should be used for. Almost all of the PO term annotations are high confidence, they are experimentally validated, and only a few of them are derived from author statements.

The strings which are described in the synonyms column are included as references to each gene, and are combined with the gene name mentioned in the symbol column into a single bar delimited list.

In [ ]:
# Reading in the dataset of spatial PO term annotations.
filename = "../data/gene_related_files/tair/po_anatomy_gene_arabidopsis_tair.assoc"
df_po_spatial = pd.read_table(filename, header=None, skiprows=1, usecols=[2,4,5,6,9,10,11])
df_po_spatial.columns = ["symbol","term_id","references","evidence_code","name","synonyms","type"]
unique_values = {col:len(pd.unique(df_po_spatial[col].values)) for col in df_po_spatial.columns}
print(df_po_spatial.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

In [ ]:
# Reading in the dataset of temporal PO term annotations.
filename = "../data/gene_related_files/tair/po_temporal_gene_arabidopsis_tair.assoc"
df_po_temporal = pd.read_table(filename, header=None, skiprows=1, usecols=[2,4,5,6,9,10,11])
df_po_temporal.columns = ["symbol","term_id","references","evidence_code","name","synonyms","type"]
unique_values = {col:len(pd.unique(df_po_temporal[col].values)) for col in df_po_temporal.columns}
print(df_po_temporal.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

In [ ]:
# Looking at how many unique values each column has.
df_po = df_po_spatial.append(df_po_temporal, ignore_index=True)
unique_values = {col:len(pd.unique(df_po[col].values)) for col in df_po.columns}
print(df_po[["symbol","synonyms","evidence_code"]].head(10))
print(df_po.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

In [ ]:
# Quantifying the number of annotations of each type.
code_quantities = {c:len([x for x in df_po["evidence_code"] if EVIDENCE_CODES[x] in c]) 
             for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

In [ ]:
# Restructuring the dataset to include all the expected column names.
df_po["species"] = "ath"
df_po["gene_names"] = np.vectorize(concatenate_with_bar_delim)(df_po["symbol"], df_po["synonyms"])
df_po["description"] = ""
df_po["term_ids"] = df_po["term_id"]
df_po = df_po[["species", "gene_names", "description", "term_ids"]]

# Outputting the dataset of annotations to a csv file.
path = os.path.join(OUTPUT_DIR,"ath_high_confidence_po_annotations.csv")
df_po.to_csv(path, index=False)